In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
import pickle

Using TensorFlow backend.


In [2]:
# convert raw .dat file to  csv using curate_db.py
# file has to exist in db/udb_1yr.dat
# !python curate_db.py 

## Import and Curate Dataframe

In [3]:
all_dat = pd.read_csv('./curated.csv', index_col=0)

# sift out pwrs
all_dat = all_dat.loc[all_dat['reactor_type'] == 'PWR']  
all_dat = sklearn.utils.shuffle(all_dat)
# only get assemblies with enrichment bigger than 1.5 and bunrup higher than 10,000
all_dat = all_dat.loc[(all_dat['init_enr'] > 1.5) & (all_dat['bu'] > 10000)]

# separate training and testing set
row_num = all_dat.shape[0]
cutoff = int(row_num * 0.6)
train_dat = all_dat.iloc[:cutoff, :]
test_dat = all_dat.iloc[cutoff:, :]


## Categorize Isotopes

In [4]:
# get all the isotopes, sorted by A
def get_name(f):
    z = ''
    for i in f:
        if i.isalpha():
            z += i
    return z

def get_a(f):
    z = ''
    for i in f:
        if i.isdigit():
            z += i
    return int(z)

iso_list = list(train_dat)[4:]
# set the isotopes to categories:
fp = []
bred_fissile = []
u235 = []
nottru = []
tru = []

for iso in iso_list:
    if iso == 'u-235':
        u235.append(iso)
    elif iso in ['pu-239', 'u-233']:
        bred_fissile.append(iso)
    elif get_a(iso) < 200:
        fp.append(iso)
    elif get_name(iso) in ['np', 'pu', 'am', 'cm']:
        tru.append(iso)
    else:
        nottru.append(iso)

category = {'u235': u235,
            'bred_fissile': bred_fissile,
            'fp': fp,
            'tru': tru,
            'nottru': nottru}

print(category)

{'u235': ['u-235'], 'bred_fissile': ['pu-239', 'u-233'], 'fp': ['zr-95', 'zr-93', 'ru-103', 'ru-106', 'sb-124', 'sb-125', 'nb-95', 'pd-107', 'pm-147', 'i-129', 'h-3', 'kr-85', 'nb-94', 'cs-134', 'cs-135', 'eu-152', 'cs-137', 'eu-154', 'eu-155', 'ag-108m', 'ag-110m', 'c-14', 'ce-144', 'y-90', 'tc-99', 'sm-151', 'se-79', 'sr-90', 'sn-126'], 'tru': ['pu-238', 'np-237', 'pu-240', 'pu-241', 'pu-244', 'pu-242', 'cm-248', 'cm-245', 'cm-244', 'cm-246', 'cm-247', 'am-241', 'am-243', 'cm-243', 'cm-242'], 'nottru': ['ra-226', 'pa-231', 'am-242m', 'cf-251', 'cf-249', 'ac-227', 'u-234', 'u-236', 'u-238', 'th-229', 'th-232', 'th-230', 'u-232']}


## Define regression algorithms

- From Sklearn:
    - linear regression
    - bayesian ridge
    - huber regressor
    - ridge
    - lasso
    - random forest
- From Keras:
    - neural network

In [5]:
algorithms = {'lin_least_square': linear_model.LinearRegression,
              'bayesianridge': linear_model.BayesianRidge,
              'huberregressor': linear_model.HuberRegressor,
              'ridge': linear_model.Ridge,
              'lasso': linear_model.Lasso}
    
def linear_regression(algorithm, xtrain, ytrain, xtest, ytest):
    al = algorithm()
    model = al.fit(xtrain, ytrain)
    model_err = (ytest - model.predict(xtest))**2
    return model, model_err

def poly_regression(xtrain, ytrain, xtest, ytest, deg=2):
    poly = sklearn.preprocessing.PolynomialFeatures(degree=deg)
    x_ = poly.fit_transform(xtrain)
    predict_ = poly.fit_transform(xtest)
    
    # remove polynomial orders that isn't necessary (optional)
    
    model = linear_model.LinearRegression()
    model.fit(x_, ytrain)
    
    prediction = model.predict(predict_)
    model_err = (ytest - prediction)**2
    return model, model_err
    

def random_forest(xtrain, ytrain, xtest, ytest,
                  estimators=1000, state=42):    
    # Instantiate model with 1000 decision trees
    model = RandomForestRegressor(n_estimators = estimators, random_state = state)
    # Train the model on training data
    model.fit(xtrain, ytrain)
    model_err = (ytest - model.predict(xtest))**2
    return model, model_err



## Find best model for each isotope
Train model where:

**features**: burnup, enrichment

**target**: composition of isotope

Find model that predicts the isotopic composition with smallest squared error for each isotope

In [ ]:
x_train = train_dat[['init_enr', 'bu']].as_matrix()
x_test = test_dat[['init_enr', 'bu']].as_matrix()
iso_err_dict = {}
alg_dict = {}

for iso in iso_list:
    print(iso)
    err_dict = {}
    alg_buff = {}
    y_train = np.asarray(train_dat[iso])
    y_test = np.asarray(test_dat[iso])
    
    for key, val in algorithms.items():
        try:
            alg_buff[key], err_dict[key] = linear_regression(val, x_train, y_train, x_test, y_test)
        except:
            print('Cant use ', key)
    alg_buff['random forest'], err_dict['random forest'] = random_forest(x_train, y_train, x_test, y_test)
    alg_buff['poly2'], err_dict['poly2'] = poly_regression(x_train, y_train, x_test, y_test, deg=2)
    alg_buff['poly3'], err_dict['poly3'] = poly_regression(x_train, y_train, x_test, y_test, deg=3)
    alg_buff['poly4'], err_dict['poly4'] = poly_regression(x_train, y_train, x_test, y_test, deg=4)
    alg_buff['poly5'], err_dict['poly5'] = poly_regression(x_train, y_train, x_test, y_test, deg=5)

    
    mean_err_dict = {}
    for key, val in err_dict.items():
        mean_err_dict[key] = np.abs(np.mean(val))
    
    chosen_alg = min(mean_err_dict, key=mean_err_dict.get)
    print(chosen_alg)
    err = err_dict[chosen_alg]
    iso_err_dict[iso] = np.mean(err)
    alg_dict[iso] = alg_buff[chosen_alg]
    print(np.mean(err))
    
    plt.scatter(x_test[:,0], x_test[:,1], c=err)
    plt.xlabel('Enrichment')
    plt.ylabel('Burnup')
    plt.colorbar()
    plt.legend()
    plt.grid()
    plt.show()
    plt.close()


## Two things:
- iso_err_dict
    - key: isotope
    - value: Mean error of model with test set
- alg_dict
    - key: isotope
    - value: fitted model object

In [ ]:
for key, val in category.items():
    err_list = []
    for iso in val:
        err_list.append(iso_err_dict[iso])
        if np.abs(iso_err_dict[iso]) > 0.05:
            print(iso, iso_err_dict[iso])
    plt.figure(figsize=(25,10))
    plt.bar(val, err_list)    
    plt.title(key)
    plt.grid()
    plt.show()
    plt.close()


## Pickle alg_dict to be used elsewhere

In [ ]:
f = open('lin_dep.pkl', 'wb')
pickle.dump(alg_dict, f)
f.close()